In [1]:
import pandas as pd

In [2]:
match_df = pd.read_csv('./datasets/IPL_Matches_2008_2022.csv')
match_df.head()

,ID,City,Date,Season,MatchNumber,Team1,Team2,Venue,TossWinner,TossDecision,SuperOver,WinningTeam,WonBy,Margin,method,Player_of_Match,Team1Players,Team2Players,Umpire1,Umpire2
0,1312200,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon
1,1312199,Ahmedabad,2022-05-27,2022,Qualifier 2,Royal Challengers Bangalore,Rajasthan Royals,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,field,N,Rajasthan Royals,Wickets,7.0,NaN,JC Buttler,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...",CB Gaffaney,Nitin Menon
2,1312198,Kolkata,2022-05-25,2022,Eliminator,Royal Challengers Bangalore,Lucknow Super Giants,"Eden Gardens, Kolkata",Lucknow Super Giants,field,N,Royal Challengers Bangalore,Runs,14.0,NaN,RM Patidar,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['Q de Kock', 'KL Rahul', 'M Vohra', 'DJ Hooda...",J Madanagopal,MA Gough
3,1312197,Kolkata,2022-05-24,2022,Qualifier 1,Rajasthan Royals,Gujarat Titans,"Eden Gardens, Kolkata",Gujarat Titans,field,N,Gujarat Titans,Wickets,7.0,NaN,DA Miller,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",BNJ Oxenford,VK Sharma
4,1304116,Mumbai,2022-05-22,2022,70,Sunrisers Hyderabad,Punjab Kings,"Wankhede Stadium, Mumbai",Sunrisers Hyderabad,bat,N,Punjab Kings,Wickets,5.0,NaN,Harpreet Brar,"['PK Garg', 'Abhishek Sharma', 'RA Tripathi', ...","['JM Bairstow', 'S Dhawan', 'M Shahrukh Khan',...",AK Chaudhary,NA Patwardhan


In [3]:
ball_by_ball_df = pd.read_csv('./datasets/IPL_Ball_by_Ball_2008_2022.csv')
ball_by_ball_df.head()

,ID,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam
0,1312200,1,0,1,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals
1,1312200,1,0,2,YBK Jaiswal,Mohammed Shami,JC Buttler,legbyes,0,1,1,0,0,NaN,NaN,NaN,Rajasthan Royals
2,1312200,1,0,3,JC Buttler,Mohammed Shami,YBK Jaiswal,NaN,1,0,1,0,0,NaN,NaN,NaN,Rajasthan Royals
3,1312200,1,0,4,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals
4,1312200,1,0,5,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals


In [4]:
motm = match_df['Player_of_Match'].value_counts()
motm

Player_of_Match
AB de Villiers    25
CH Gayle          22
DA Warner         18
RG Sharma         18
MS Dhoni          17
                  ..
CR Brathwaite      1
Mohsin Khan        1
A Zampa            1
BCJ Cutting        1
MF Maharoof        1
Name: count, Length: 262, dtype: int64

In [5]:
# Filter relevant columns for bowler analysis
bowler_data = ball_by_ball_df[['ID', 'bowler', 'total_run', 'extra_type', 'isWicketDelivery', 'overs', 'kind']]

# Aggregate data by bowler
bowler_agg = bowler_data.groupby('bowler').agg(
    matches=('ID', 'nunique'),
    total_runs_conceded=('total_run', 'sum'),
    total_balls_bowled=('total_run', 'count'),
    total_illegal_deliveries=('extra_type', lambda x: ((x == 'noballs') | (x == 'wides') | (x == 'penalty')).sum()),
    total_wides=('extra_type', lambda x: (x == 'wides').sum()),
    total_no_balls=('extra_type', lambda x: (x == 'noballs').sum()),
).reset_index()

bowler_agg['total_legal_deliveries'] = bowler_agg['total_balls_bowled'] - bowler_agg['total_illegal_deliveries']
bowler_agg['avg_runs_conceded'] = bowler_agg['total_runs_conceded']/bowler_agg['matches']
# find total wickets which are not run out
wicket_data = bowler_data[(bowler_data['isWicketDelivery'] == 1) & (bowler_data['kind'] != 'run out')]

# Aggregate data by bowler
wicket_agg = wicket_data.groupby('bowler').agg(
    total_wickets=('isWicketDelivery', 'sum'),
    bowled=('kind', lambda x: (x == 'bowled').sum()),
    lbw=('kind', lambda x: (x == 'lbw').sum()),
    caught=('kind', lambda x: (x == 'caught').sum()),
    stumped=('kind', lambda x: (x == 'stumped').sum()),
    others=('kind', lambda x: ((x != 'bowled') & (x != 'lbw') & (x != 'caught') & (x != 'stumped')).sum()),
).reset_index()

# Merge wicket and run data
bowler_agg = bowler_agg.merge(wicket_agg, on='bowler', how='left')

bowler_agg['total_wickets'] = bowler_agg['total_wickets'].fillna(0).astype(int)

In [6]:
bowler_overs = ball_by_ball_df.groupby('bowler').apply(lambda x: x[['ID', 'overs']].drop_duplicates().shape[0]).reset_index(name='total_overs')
bowler_overs

,bowler,total_overs
0,A Ashish Reddy,45
1,A Badoni,2
2,A Chandila,39
3,A Choudhary,17
4,A Dananjaya,4
...,...,...
467,YK Pathan,193
468,YS Chahal,480
469,Yash Dayal,32
470,Yuvraj Singh,146


In [7]:
# map bowler overs
bowler_agg = bowler_agg.merge(bowler_overs, on='bowler')

In [8]:
# Calculate additional metrics for bowlers
bowler_agg['economy'] = bowler_agg['total_runs_conceded'] / (bowler_agg['total_overs'])

In [9]:
bowler_agg['motm_count'] = bowler_agg['bowler'].map(motm).fillna(0).astype(int)

In [10]:
bowler_agg[bowler_agg['bowler'].str.contains('Tanvir')]

,bowler,matches,total_runs_conceded,total_balls_bowled,total_illegal_deliveries,total_wides,total_no_balls,total_legal_deliveries,avg_runs_conceded,total_wickets,bowled,lbw,caught,stumped,others,total_overs,economy,motm_count
425,Sohail Tanvir,11,275,265,18,17,1,247,25.0,22,8.0,3.0,11.0,0.0,0.0,42,6.547619,2


In [11]:
bowler_agg.loc[bowler_agg['bowler'].str.contains('YS Chahal'), 'purple_caps'] = 1
bowler_agg.loc[bowler_agg['bowler'].str.contains('HV Patel'), 'purple_caps'] = 1
bowler_agg.loc[bowler_agg['bowler'].str.contains('K Rabada'), 'purple_caps'] = 1
bowler_agg.loc[bowler_agg['bowler'].str.contains('Imran Tahir'), 'purple_caps'] = 1
bowler_agg.loc[bowler_agg['bowler'].str.contains('AJ Tye'), 'purple_caps'] = 1
bowler_agg.loc[bowler_agg['bowler'].str.contains('B Kumar'), 'purple_caps'] = 2
bowler_agg.loc[bowler_agg['bowler'].str.contains('DJ Bravo'), 'purple_caps'] = 2
bowler_agg.loc[bowler_agg['bowler'].str.contains('MM Sharma'), 'purple_caps'] = 1
bowler_agg.loc[bowler_agg['bowler'].str.contains('M Morkel'), 'purple_caps'] = 1
bowler_agg.loc[bowler_agg['bowler'].str.contains('SL Malinga'), 'purple_caps'] = 1
bowler_agg.loc[bowler_agg['bowler'].str.contains('PP Ojha'), 'purple_caps'] = 1
bowler_agg.loc[bowler_agg['bowler'].str.contains('RP Singh'), 'purple_caps'] = 1
bowler_agg.loc[bowler_agg['bowler'].str.contains('Sohail Tanvir'), 'purple_caps'] = 1
bowler_agg['purple_caps'] = bowler_agg['purple_caps'].fillna(0).astype(int)

In [12]:
bowler_agg[bowler_agg['total_overs'] >= 20].sort_values(by='purple_caps', ascending=False).head(10)

,bowler,matches,total_runs_conceded,total_balls_bowled,total_illegal_deliveries,total_wides,total_no_balls,total_legal_deliveries,avg_runs_conceded,total_wickets,bowled,lbw,caught,stumped,others,total_overs,economy,motm_count,purple_caps
63,B Kumar,146,4064,3384,122,111,11,3262,27.835616,154,36.0,9.0,103.0,1.0,5.0,549,7.402550,5,2
109,DJ Bravo,158,4436,3296,176,167,9,3120,28.075949,183,21.0,8.0,143.0,0.0,11.0,527,8.417457,4,2
344,RP Singh,82,2417,1874,99,86,13,1775,29.475610,90,25.0,5.0,58.0,0.0,2.0,299,8.083612,1,1
148,HV Patel,76,2262,1665,80,70,10,1585,29.763158,97,10.0,7.0,78.0,0.0,2.0,268,8.440299,4,1
311,PP Ojha,90,2399,1945,47,44,2,1898,26.655556,89,15.0,5.0,57.0,12.0,0.0,318,7.544025,1,1
425,Sohail Tanvir,11,275,265,18,17,1,247,25.000000,22,8.0,3.0,11.0,0.0,0.0,42,6.547619,2,1
393,SL Malinga,122,3486,2974,147,128,18,2827,28.573770,170,63.0,11.0,90.0,0.0,6.0,478,7.292887,6,1
196,K Rabada,63,2023,1498,60,51,9,1438,32.111111,102,15.0,0.0,83.0,0.0,4.0,241,8.394191,2,1
160,Imran Tahir,59,1729,1340,24,23,1,1316,29.305085,82,16.0,7.0,51.0,3.0,5.0,221,7.823529,1,1
265,MM Sharma,86,2488,1795,41,35,6,1754,28.930233,92,21.0,4.0,65.0,0.0,2.0,297,8.377104,2,1


In [13]:
bowler_agg.to_csv('./datasets/bowler_agg.csv', index=False)